## Домашнее задание по неделе 4

Как было рассказано на лекции, стохастический градиентый спуск сходится быстрее, чем полный, хотя и менее стабильно. В этом задании вам предлагается реализовать стохастический градиентный спуск и сравнить его с точным вычислением весов линейной модели по скорости работы и значению метрики качества.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings

np.random.seed(0)

warnings.filterwarnings('ignore')
%matplotlib inline

### Задание 0

Реализуйте класс ```LinearRegressionSGD``` c обучением и и применением линейной регрессии, построенной с помощью стохастического градиентного спуска, с заданным интерфейсом.

In [19]:
from sklearn.base import BaseEstimator

class LinearRegression(BaseEstimator):
    def __init__(self, epsilon=1e-4, max_steps=1000, w0=None, alpha=1e-2):
        """
        epsilon: разница для нормы изменения весов 
        max_steps: максимальное количество шагов в градиентном спуске
        w0: np.array (d,) - начальные веса
        alpha: шаг обучения
        """
        self.epsilon = epsilon
        self.max_steps = max_steps
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.w_history = []
    
    def fit(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: self
        """
        l, d = X.shape

        if self.w0 is None:
            self.w0 = np.zeros(d)

        self.w = self.w0

        for step in range(self.max_steps):
            self.w_history.append(self.w)

            w_new = self.w - self.alpha * self.calc_gradient(X, y)

            if (np.linalg.norm(w_new - self.w) < self.epsilon):
                break
          
            self.w = w_new
        
        return self
    
    def predict(self, X):
        """
        X: np.array (l, d)
        ---
        output: np.array (l)
        """
        
        if self.w is None:
            raise Exception('Not trained yet')
        
        l, d = X.shape

        y_pred = []

        for i in range(l):
            y_pred.append(np.dot(X[i], self.w))

        return np.array(y_pred)
    
    def calc_gradient(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: np.array (d)
        """
        
        l, d = X.shape
        gradient = []
        
        for j in range(d):
            dQ = 0
            for i in range(l):
                dQ += (2/l) * X[i][j] * (np.dot(X[i], self.w) - y[i])
            gradient.append(dQ)

        return np.array(gradient)

In [27]:
from sklearn.base import BaseEstimator

from sklearn.linear_model import SGDRegressor

class LinearRegressionSGD(BaseEstimator):
    def __init__(self, epsilon=1e-4, max_steps=100, w0=None, alpha=1e-4):
        """
        epsilon: разница для нормы изменения весов 
        max_steps: максимальное количество шагов в градиентном спуске
        w0: np.array (d,) - начальные веса
        alpha: шаг обучения
        """
        self.epsilon = epsilon
        self.max_steps = max_steps
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.w_history = []
    
    def fit(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: self
        """
        l, d = X.shape
        
        X = X[:1, :]

        if self.w0 is None:
            self.w0 = np.zeros(d)

        self.w = self.w0

        for step in range(self.max_steps):
            self.w_history.append(self.w)

            w_new = self.w - self.alpha * self.calc_gradient(X, y)

            if (np.linalg.norm(w_new - self.w) < self.epsilon):
                break
          
            self.w = w_new
        
        return self
        
    
    def predict(self, X):
        """
        X: np.array (l, d)
        ---
        output: np.array (l)
        """
        if self.w is None:
            raise Exception('Not trained yet')
        
        l, d = X.shape

        y_pred = []

        for i in range(l):
            y_pred.append(np.dot(X[i], self.w))

        return np.array(y_pred)
    
    def calc_gradient(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: np.array (d)
        """
        l, d = X.shape
        gradient = []
        
        for j in range(d):
            dQ = 0
            for i in range(l):
                dQ += (2/l) * X[i][j] * (np.dot(X[i], self.w) - y[i])
            gradient.append(dQ)

        return np.array(gradient)

Проверять работу мы будем на имеющемся в sklearn наборе данных boston: в нём нужно по информации о доме предсказать его стоимость.

In [2]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

data = load_boston()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

X_train, X_test, y_train, y_test = train_test_split(np.array(X), y, test_size=0.3, random_state=10)

### Задание 1

Метрикой качества в нашей задаче будет MAPE - Mean Absolute Percentage Error. Реализуйте её с заданным интефейсом и вычислите 
```MAPE(y_test, y_0)```, где ```y_0 = (mean(y_test), mean(y_test), ...)```

In [9]:
def MAPE(y_true, y_pred):
    """
        y_true: np.array (l)
        y_pred: np.array (l)
        ---
        output: float [0, +inf)
    """
    #from sklearn.utils import check_arrays не импортирует чего-то
    def mean_absolute_percentage_error(y_true, y_pred): 
        #y_true, y_pred = check_arrays(y_true, y_pred)

    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return mean_absolute_percentage_error(y_true, y_pred)

In [14]:
y_0 = np.array([np.mean(y_test) for i in y_test])
print(y_0)
print(y_test)
MAPE(y_test, y_0)

[23.84078947 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947
 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947
 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947
 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947
 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947
 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947
 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947
 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947
 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947
 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947
 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947
 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947
 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947 23.84078947
 23.84078947 23.84078947 23.84078947 23.84078947 23

37.41588297684096

### Задание 2 

Обучите ```LinearRegressionSGD``` с базовыми параметрами на тренировочном наборе данных (```X_train```, ```y_train```), сделайте предсказание на тестовых данных ```X_test```, записав результат в переменную ```y_pred_sgd```  и вычислите ошибку MAPE.

In [28]:
sgd = LinearRegressionSGD().fit(X_train, y_train)

y_pred_sgd = sgd.predict(X_test)
print(y_pred_sgd)
MAPE(y_test, y_pred_sgd)

[-5.60114989e+168 -5.80636786e+168 -6.35788461e+168 -6.30722568e+168
 -6.42215043e+168 -5.81757784e+168 -6.73212290e+168 -7.20526483e+168
 -5.59505765e+168 -6.18647702e+168 -6.72492631e+168 -6.20022008e+168
 -5.44214525e+168 -5.82911161e+168 -5.76566559e+168 -5.88975549e+168
 -5.97387782e+168 -5.43639883e+168 -6.09290107e+168 -6.20900607e+168
 -6.63492544e+168 -8.01260655e+168 -7.06576615e+168 -6.05477672e+168
 -6.73394683e+168 -5.62890737e+168 -6.32843504e+168 -7.53008797e+168
 -5.37853764e+168 -5.12814363e+168 -5.91108884e+168 -6.02097149e+168
 -6.43077816e+168 -5.58971754e+168 -8.78953350e+168 -6.83713262e+168
 -5.50688534e+168 -5.73840991e+168 -6.19667197e+168 -5.96658270e+168
 -5.95467644e+168 -6.27096953e+168 -8.60086523e+168 -5.65657216e+168
 -5.72498167e+168 -6.66236755e+168 -8.32853138e+168 -8.64923823e+168
 -5.50937207e+168 -6.27319146e+168 -6.94163912e+168 -8.80103705e+168
 -6.06144554e+168 -5.93713581e+168 -6.01779992e+168 -5.93072139e+168
 -7.05725568e+168 -8.71057042e+168

3.296541086556417e+169

In [30]:
sgd = SGDRegressor().fit(X_train, y_train)

y_pred_sgd = sgd.predict(X_test)
print(y_pred_sgd)
MAPE(y_test, y_pred_sgd)

[4.79905837e+13 2.72725754e+13 2.79955170e+13 4.50611862e+13
 9.15869584e+13 9.03920144e+13 8.61998939e+13 9.83532513e+13
 5.75803054e+13 7.31651269e+13 8.37618222e+13 7.40857401e+13
 5.96313925e+13 4.32600886e+13 3.94890017e+13 6.47115733e+13
 5.37434460e+13 6.86628915e+13 6.46408406e+13 8.97416265e+13
 8.85716643e+13 1.08732064e+14 8.71279638e+13 6.12079207e+13
 4.68070929e+13 5.87112911e+13 8.03817743e+13 1.00979917e+14
 4.74987471e+13 9.27611289e+13 3.08040229e+13 7.18184989e+13
 3.01675655e+13 4.85747810e+13 1.14323865e+14 8.20298561e+13
 9.34223649e+13 2.28865063e+13 6.70901824e+13 6.82232124e+13
 5.41562576e+13 7.44105411e+13 1.13955118e+14 2.82483347e+13
 7.45340957e+13 5.10204214e+13 1.08844483e+14 1.06902855e+14
 5.24467519e+13 8.12906728e+13 9.22276536e+13 1.15030276e+14
 7.15194692e+13 5.61964659e+13 7.46253818e+13 6.43022640e+13
 8.36441000e+13 1.08341397e+14 6.22292228e+13 1.08947204e+14
 9.63830067e+13 8.49938828e+13 8.85658099e+13 6.32478940e+13
 5.85582549e+13 9.925342

391769752150773.2

### Задание 3

Вычислите веса по точной формуле, используя ```X_train``` и ```y_train```; предскажите с их помощью целевую переменную на ```X_test```, записав результат в переменную ```y_pred_lr``` и вычислите ошибку MAPE.

In [31]:
from sklearn.linear_model import LinearRegression

y_pred_lr = LinearRegression().fit(X_train, y_train).predict(X_test)

MAPE(y_test, y_pred_lr)

17.391972849542015

## Бонусное задание по неделе 4

До этого вы релизовывали модели, в которых не было штрафа за величину весов ```w```. Как было рассказано ранее в лекциях, это может привести к неустойчивости модели и переобучению. Чтобы избежать этих эффектов, предлагается добавить к оптимизируемому функционалу L2-норму весов; таким образом, будем решать задачу гребневой регрессии, Ridge:

$$ \frac{1}{l}(Xw-y)^T(Xw-y) +\gamma||w||_2 \rightarrow \min_{w}. $$

### Задание 11
Реализуйте обучение такой модели в матричном виде (смотрите дополнительные материалы к этой неделе) с помощью стохастического градиентного спуска. Класс должен совпадать по набору реализованных функций с ```LinearRegressionVectorized```, разница будет лишь в параметре $\gamma$, отвечающем за степень регуляризации. 

In [0]:
class RidgeSGD(BaseEstimator):
    def __init__(self, epsilon=1e-4, max_steps=1000, w0=None, alpha=1e-2, gamma=0):
        """
        epsilon: разница для нормы изменения весов 
        max_steps: максимальное количество шагов в градиентном спуске
        w0: np.array (d,) - начальные веса
        alpha: шаг обучения
        gamma: коэффициент регуляризации
        """
        self.epsilon = epsilon
        self.max_steps = max_steps
        self.w0 = w0
        self.alpha = alpha
        self.gamma = gamma
        self.w = None
        self.w_history = []
    
    def fit(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: self
        """
        return self
    
    def predict(self, X):
        """
        X: np.array (l, d)
        ---
        output: np.array (l)
        """
        pass

    
    def calc_gradient(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: np.array (d)
        """
        pass

Так же, как и в основном задании, обучите модель с базовыми параметрами на тренировочных данных и сделайте прогноз y_pred_ridge. Выведите значение MAPE(y_test, y_pred_ridge).